In [ ]:
import numpy as np
import pandas as pd
from datetime import datetime
import os
import matplotlib.pyplot as plt
import matplotlib as mpl


In [ ]:
df = pd.read_csv('data/data.txt')
df['Start'] = pd.to_datetime(df['Start'])
df['End'] = pd.to_datetime(df['End'])
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]  # удалаяем битые столбцы
df['Total'] = df['Total'].str.extract(
    '(\d+)').astype(int)  # удаляем из Total 'min'

print(df.info())


In [ ]:
activities = df.Activity.unique()

# first_year = df.Start.min().year
# last_year = df.End.max().year

# for year in range(first_year, last_year + 1):
year = 2022

weeks = range(1, 54)
dfy = pd.DataFrame(0, index=np.arange(len(weeks)),
                   columns=np.hstack(['week', activities]))
dfy['week'] = weeks
dfy = dfy.set_index('week')

for index, row in df.iterrows():
    if (row.Start.year > year):
        continue
    if (row.Start.year < year):
        break

    dfy[row.Activity].iloc[row.Start.week - 1] += row.Total

dfy = dfy.loc[:, (dfy != 0).any(axis=0)]  # удаляем столбцы, в которых все нули


In [ ]:
cdf = pd.read_csv('data/colors.txt')
color_dict = dict(cdf.values)
color_dict


In [ ]:
colors = []
for c in dfy.columns:
    if c in color_dict:
        colors.append(color_dict[c])
    else:
        colors.append("orange")
colors


In [ ]:
dfy.plot(kind='area', stacked=False, figsize=(20, 10), color=colors, linewidth=0)
plt.title(year)
plt.ylabel('min')
plt.xlabel('week')
plt.show()


In [ ]:
os.makedirs('data', exist_ok=True)
dfy.to_csv('data/' + str(year) + '.csv', index=False)
